<div class='bar_title'></div>

*Enterprise AI*

# Assignment 4 - Experiment Tracking and Model Deployment

Gunther Gust / Justus Ameling<br>
Chair of Enterprise AI

Summer Semester 2024

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:20%; float:left;" />

In [ ]:
!zenml init

As mentioned in the last assignments, ZenML is a tool that allows the creation of ML pipelines without creating a lock-in effect, since hardware and software are decoupled. This is achieved by two main concepts: the pipeline itself and the stack. The pipeline is defined in Python files and represents the sequence of tasks to be executed. On the other hand, the stack represents the additional tools and infrastructure used. While we already have a general understanding of the pipeline structure, this time, we will explore stacks a bit more.

Stacks in ZenML consist of several components. A component is a representation of a configuration of a particular function. Two components that are always required in your stack, and therefore already used by you, are the orchestrator and the artifact store.

**The Orchestrator:**
- The Orchestrator is something like the control center
- It decides when and how steps in your pipeline should be executed.

**The Artifact Store:**
- It stores our artifacts, which are the in and outputs of our pipelines.
- By default, we use a local artifact store.

Feel free to look at the ZenMl documentation to learn more about stacks and components <a href="https://docs.zenml.io/user-guide/production-guide/understand-stacks">HERE</a> and  <a href="https://docs.zenml.io/stacks-and-components/component-guide">HERE</a> 

<img src="./images/stacks.avif" style="width:20%;" />

When we want to see our current stack, we can simply run the following command: `zenml stack describe`. This will show us the current stack configuration. 

In [ ]:
!zenml stack describe

As you can see, the command returns as a table showing the selected artifact store and orchestrator. Both are, at the moment, the default version. 

In ZenML, there are various solutions called "flavors" for different components. For this task, we need to create and add two new components: one for experiment tracking and one for model deployment. Both are of the Flavor Mlflow. 

First of all, we need to install the MLflow package with the suitable version

In [ ]:
!zenml integration install mlflow -y

Next, we want to create the component that we want to add to the stack. 
Let us begin with the Experiment Tracker:

In [ ]:
!zenml experiment-tracker register mlflow_experiment_tracker --flavor=mlflow

Let us break down the command:
- experiment-tracker: This is the selection of the top-level group, so the type of component
- register: indicating that we want to register a new component
- mlfow_experiment_tracker: This is a unique name that our component should have
- --flavor=mlflow: Here, we are selecting the implementation of the component, in our case MLflow

We need to do the same with the Model Deployment component.
So let us execute the following command to create the Model Deployment component:

In [ ]:
!zenml model-deployer register mlflow_deployer --flavor=mlflow

Now that we have created the components, we need to add them to the stack. This is done by executing the following command:

In [ ]:
!zenml stack register mlflow_stack -e mlflow_experiment_tracker -d mlflow_deployer -o default -a default --set

Let us also drill down this command:
stack: we are modifying the stack
register: indicating that we want to add a new stack
mlflow_stack: The unique name that helps us to identify the stack
- -e indicates that we want to add an experiment tracker with the name "mlfow_experiment_tracker"
- -d: indicates that we want to add a model deployer with the name "mlflow_deployer"
- -o: indicates that we want to add the default orchestrator
- -a: indicates that we want to add the default artifact store

Parameters -o and -a are always mandatory when creating a stack. 

Hint: We can see additional options by calling `!zenml --help`.

Finally, we can check if the stack has been created successfully by running the following command:

In [ ]:
!zenml stack describe

With the stack of our ZenML pipeline set up, we can take a look at the code. In this assignment, you will find three pipelines that make up our ML system. First, you will find a feature engineering pipeline. Second, you can find a training pipeline that builds our model, and finally, you can find an inference pipeline that uses a deployed model to predict the weather forecast for the next day in Perth.

Before you can run the ML system, you need to fix some parts of the code, first look at the feature_engineering_pipeline, then look at the training_pipeline, and finally look at the inference_pipeline.

Finally, to run your complete pipeline or ML system, you can use the !python run.py command in your Jupyter notebook or python run.py in your command line.

In [ ]:
!python run.py

After successfully executing the pipeline, we want to take a look at our experiment tracking dashboard, which MLflow provides. Therefore, we have to take the following two steps:
- First, we need to get the URL or path to the location where our experiment tracker data is stored. This is done by executing the first code cell below.
- Second, we need to start the dashboard server by executing the second cell of the following code

In [ ]:
from zenml.client import Client
pipeline_run = Client().get_pipeline("training_pipeline").last_run
step = pipeline_run.steps["model_trainer"]
experiment_tracker_url = step.run_metadata["experiment_tracker_url"].value
experiment_tracker_url

In [ ]:
!mlflow ui --backend-store-uri ########PLEASE REPLACE THIS WITH THE EXPERIMENT TRACKER URL from the code cell above########

Use the MLflow dashboard to review the metrics of the last run.  Please explain the meaning of the different metrics, including precision, recall, and F1 score.

- True Positives (TP): The number of positive cases correctly predicted by the model.
- True Negatives (TN): The number of negative cases correctly predicted by the model.
- False Positives (FP): The number of negative cases incorrectly classified as positive.
- False Negatives (FN): The number of positive cases incorrectly classified as negative.

Precision = $\frac{TP}{TP + FP}$<br>
Precision is the ratio of correctly predicted positive observations to the total predicted positive observations.

Recall = $\frac{TP}{TP + FN}$<br>
Recall is the ratio of correctly predicted positive observations to the all observations in actual class.

F1 Score = $2 \times \frac{Precision \times Recall}{Precision + Recall}$<br>
F1 Score is the weighted average of Precision and Recall.

